# Notebook Summary

This notebook demonstrates how to create a sample Spark DataFrame containing WKT (Well-Known Text) geometries and perform spatial queries using Databricks SQL spatial functions. The workflow includes:

- Creating a DataFrame with point geometries in WKT format.
- Registering the DataFrame as a temporary SQL view.
- Defining a polygon in WKT format for spatial filtering.
- Using spatial SQL functions (`ST_Within`, `ST_Distance`, `ST_GeomFromWKT`) to:
  - Filter points within the polygon.
  - Calculate the distance of each point from the origin.

**Note:**  
Spatial SQL functions require Databricks Runtime 17.1 (Beta) or later. (Photon is optional)

Documentation:

- [Medium Overview](https://medium.com/towards-data-engineering/databricks-spatial-sql-is-here-a-game-changer-for-geospatial-intelligence-on-the-lakehouse-e7b1e5eb50ba)
- [Mosaic Spatial SQL](https://databrickslabs.github.io/mosaic/api/spatial-functions.html)

In [0]:

# Create a sample DataFrame with WKT geometries
data = [
    (i, f"POINT({i} {i})") for i in range(10)
]
columns = ["id", "wkt"]

df = spark.createDataFrame(data, columns)

# Register as a temp view
df.createOrReplaceTempView("points")

# Create a polygon for spatial queries
polygon_wkt = "POLYGON((0 0, 0 5, 5 5, 5 0, 0 0))"

# Use Spatial SQL functions to filter points within the polygon
result = spark.sql(f"""
SELECT
  id,
  wkt,
  ST_Within(ST_GeomFromWKT(wkt), ST_GeomFromWKT('{polygon_wkt}')) AS within_polygon,
  ST_Distance(ST_GeomFromWKT(wkt), ST_GeomFromWKT('POINT(0 0)')) AS distance_from_origin
FROM points
""")

display(result)

In [0]:
# Create a sample DataFrame with WKT lines
data = [
    (0, "LINESTRING(1 1, 4 4)"),
    (1, "LINESTRING(6 6, 8 8)"),
    (2, "LINESTRING(2 0, 2 6)"),
    (3, "LINESTRING(0 0, 5 0)"),
    (4, "LINESTRING(6 6, 10 10)")
]
columns = ["id", "wkt"]

df = spark.createDataFrame(data, columns)

# Register as a temp view
df.createOrReplaceTempView("lines")

# Create a polygon for spatial queries
polygon_wkt = "POLYGON((0 0, 0 5, 5 5, 5 0, 0 0))"

# Use Spatial SQL functions to check which lines intersect the polygon
result = spark.sql(f"""
SELECT
  id,
  wkt,
  ST_GeomFromWKT(wkt) as line,
  ST_Intersects(ST_GeomFromWKT(wkt), ST_GeomFromWKT('{polygon_wkt}')) AS intersects_polygon
FROM lines
""")

display(result)